In [17]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

In [16]:
SparkContext.version

In [18]:
spark = SparkSession.builder \
        .appName('kafka') \
        .getOrCreate()

In [19]:
stream_df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "broker:29092") \
  .option("startingOffsets", "earliest") \
  .option("subscribe", "submissions") \
  .load()

In [20]:
stream_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [21]:
raw_stream = stream_df \
    .writeStream \
    .format("memory") \
    .queryName("raw_submission_view") \
    .start()

In [22]:
from IPython.display import display, clear_output
import time

In [23]:
clear_output(wait=True)

In [25]:
clear_output(wait=True)
display(spark.sql('SELECT * FROM raw_submission_view').show(20))
time.sleep(1)

+-------------------+--------------------+-----------+---------+------+--------------------+-------------+
|                key|               value|      topic|partition|offset|           timestamp|timestampType|
+-------------------+--------------------+-----------+---------+------+--------------------+-------------+
|[6E 73 6D 66 74 35]|[7B 22 69 64 22 3...|submissions|        0|     0|2021-06-06 03:07:...|            0|
|[6E 73 6D 6A 6C 7A]|[7B 22 69 64 22 3...|submissions|        0|     1|2021-06-06 03:07:...|            0|
|[6E 73 6D 72 79 6A]|[7B 22 69 64 22 3...|submissions|        0|     2|2021-06-06 03:07:...|            0|
|[6E 73 6E 39 33 6B]|[7B 22 69 64 22 3...|submissions|        0|     3|2021-06-06 03:07:...|            0|
|[6E 73 6E 67 63 65]|[7B 22 69 64 22 3...|submissions|        0|     4|2021-06-06 03:07:...|            0|
|[6E 73 6E 78 72 6F]|[7B 22 69 64 22 3...|submissions|        0|     5|2021-06-06 03:07:...|            0|
|[6E 73 6E 79 32 67]|[7B 22 69 64 22 

None

In [26]:
raw_stream.stop()

In [27]:
from pyspark.sql.types import StringType

In [28]:
string_stream_df = stream_df \
    .withColumn("key", stream_df["key"].cast(StringType())) \
    .withColumn("value", stream_df["value"].cast(StringType()))

In [29]:
string_stream_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [30]:
string_stream = string_stream_df \
    .writeStream \
    .format("memory") \
    .queryName("string_submission_view") \
    .start()

In [31]:
clear_output(wait=True)
display(spark.sql('SELECT value FROM string_submission_view WHERE key IS NOT NULL').show(20, False))
time.sleep(1)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

None

In [32]:
string_stream.stop()

In [33]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, BooleanType, IntegerType, DoubleType, StringType

In [34]:
schema_inventory = StructType([
    StructField("author_fullname", StringType(),  True),
    StructField("title", StringType(),  True),
    StructField("subreddit_name_prefixed", StringType(), True),
    StructField("name", StringType(),  True),
    StructField("upvote_ratio", DoubleType(), True),
    StructField("ups", IntegerType(), True),
    StructField("score", IntegerType(), True),
    StructField("author_premium", BooleanType(), True),
    StructField("created", IntegerType(), True),
    StructField("domain", StringType(),  True),
    StructField("url_overridden_by_dest", StringType(),  True),
    StructField("over_18", BooleanType(), True),
    StructField("subreddit_id", StringType(),  True),
    StructField("permalink", StringType(),  True),
    StructField("parent_whitelist_status", StringType(),  True),
    StructField("url", StringType(),  True),
    StructField("created_utc", IntegerType(), True)
])


In [35]:
json_stream_df = string_stream_df\
    .withColumn("value", F.from_json("value", schema_inventory))

In [36]:
json_stream_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: struct (nullable = true)
 |    |-- author_fullname: string (nullable = true)
 |    |-- title: string (nullable = true)
 |    |-- subreddit_name_prefixed: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- upvote_ratio: double (nullable = true)
 |    |-- ups: integer (nullable = true)
 |    |-- score: integer (nullable = true)
 |    |-- author_premium: boolean (nullable = true)
 |    |-- created: integer (nullable = true)
 |    |-- domain: string (nullable = true)
 |    |-- url_overridden_by_dest: string (nullable = true)
 |    |-- over_18: boolean (nullable = true)
 |    |-- subreddit_id: string (nullable = true)
 |    |-- permalink: string (nullable = true)
 |    |-- parent_whitelist_status: string (nullable = true)
 |    |-- url: string (nullable = true)
 |    |-- created_utc: integer (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable 

In [39]:
json_stream = json_stream_df \
    .writeStream \
    .format("memory") \
    .queryName("extract_submissions") \
    .start()

IllegalArgumentException: Cannot start query with name extract_submissions as a query with that name is already active in this SparkSession

In [40]:
clear_output(wait=True)
display(spark.sql('SELECT * FROM extract_submissions').show(20))
time.sleep(1)

+------+--------------------+-----------+---------+------+--------------------+-------------+
|   key|               value|      topic|partition|offset|           timestamp|timestampType|
+------+--------------------+-----------+---------+------+--------------------+-------------+
|nsmft5|{t2_68s86ieu, Den...|submissions|        0|     0|2021-06-06 03:07:...|            0|
|nsmjlz|{t2_1p03z15q, G7:...|submissions|        0|     1|2021-06-06 03:07:...|            0|
|nsmryj|{t2_b8m8x8pu, Boa...|submissions|        0|     2|2021-06-06 03:07:...|            0|
|nsn93k|{t2_pw53y, Electi...|submissions|        0|     3|2021-06-06 03:07:...|            0|
|nsngce|{t2_4z821hqs, Chi...|submissions|        0|     4|2021-06-06 03:07:...|            0|
|nsnxro|{t2_4de6hm5l, Rar...|submissions|        0|     5|2021-06-06 03:07:...|            0|
|nsny2g|{t2_h3tqs, Anothe...|submissions|        0|     6|2021-06-06 03:07:...|            0|
|nsp54j|{t2_6bfrbxoa, Can...|submissions|        0|     7|20

None

In [41]:
json_stream.stop()

In [42]:
submission_stream_df = json_stream_df \
    .select( \
        F.col("key").alias("event_key"), \
        F.col("topic").alias("event_topic"), \
        F.col("timestamp").alias("event_timestamp"), \
            "value.author_fullname", \
            "value.title", \
            "value.subreddit_name_prefixed", \
            "value.name", \
            "value.upvote_ratio", \
            "value.ups", \
            "value.score", \
            "value.author_premium", \
            "value.created", \
            "value.domain", \
            "value.url_overridden_by_dest", \
            "value.over_18", \
            "value.subreddit_id", \
            "value.permalink", \
            "value.parent_whitelist_status", \
            "value.url", \
            "value.created_utc"
           )

In [43]:
submission_stream_df.printSchema()

root
 |-- event_key: string (nullable = true)
 |-- event_topic: string (nullable = true)
 |-- event_timestamp: timestamp (nullable = true)
 |-- author_fullname: string (nullable = true)
 |-- title: string (nullable = true)
 |-- subreddit_name_prefixed: string (nullable = true)
 |-- name: string (nullable = true)
 |-- upvote_ratio: double (nullable = true)
 |-- ups: integer (nullable = true)
 |-- score: integer (nullable = true)
 |-- author_premium: boolean (nullable = true)
 |-- created: integer (nullable = true)
 |-- domain: string (nullable = true)
 |-- url_overridden_by_dest: string (nullable = true)
 |-- over_18: boolean (nullable = true)
 |-- subreddit_id: string (nullable = true)
 |-- permalink: string (nullable = true)
 |-- parent_whitelist_status: string (nullable = true)
 |-- url: string (nullable = true)
 |-- created_utc: integer (nullable = true)



In [44]:
submission_stream = submission_stream_df \
    .writeStream \
    .format("memory") \
    .queryName("submission_view") \
    .start()

In [45]:
clear_output(wait=True)
display(spark.sql('SELECT * FROM submission_view').show(20))
time.sleep(1)

+---------+-----------+--------------------+---------------+--------------------+-----------------------+---------+------------+-----+-----+--------------+-------+--------------------+----------------------+-------+------------+--------------------+-----------------------+--------------------+-----------+
|event_key|event_topic|     event_timestamp|author_fullname|               title|subreddit_name_prefixed|     name|upvote_ratio|  ups|score|author_premium|created|              domain|url_overridden_by_dest|over_18|subreddit_id|           permalink|parent_whitelist_status|                 url|created_utc|
+---------+-----------+--------------------+---------------+--------------------+-----------------------+---------+------------+-----+-----+--------------+-------+--------------------+----------------------+-------+------------+--------------------+-----------------------+--------------------+-----------+
|   nsmft5|submissions|2021-06-06 03:07:...|    t2_68s86ieu|Denmark parliamen..

None

In [46]:
clear_output(wait=True)
display(spark.sql('SELECT domain, COUNT(1) AS count FROM submission_view GROUP BY domain ORDER BY count DESC').show(5))
time.sleep(1)

+---------------+-----+
|         domain|count|
+---------------+-----+
|    reuters.com|   16|
|theguardian.com|    6|
|        bbc.com|    6|
|           null|    5|
|      bbc.co.uk|    5|
+---------------+-----+
only showing top 5 rows



None

In [47]:
submission_stream.stop()

In [48]:
spark.stop()